<a href="https://colab.research.google.com/github/aniketjivani/generative_experiments/blob/master/ExploreHamiltonians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My somewhat silly attempt at a self-contained tutorial 🕵 on NNs + conservations laws + "real" applications. Hope this is useful! Things that are unclear to me or where potential connections need more investigation are highlighted in red, and there's plenty of these.

Hamiltonian mechanics is a branch of physics concerned with conservation laws and invariances. There is considerable interest in using deep neural networks to model dynamical systems, whether for system identification or long-term prediction purposes. Existing neural networks that are used to model dynamical systems, irrespective of whether they predict states in discrete timesteps or in a continuous fashion, do not learn exact conservation laws or invariant quantities - to quote one of the papers below, they may drift away from the true dynamics of the system as small errors accumulate. The methods we will see next, namely:

One motivating work for these is the section on Hyperbolic CNNs from Ruthotto and Haber (https://doi.org/10.1007/s10851-019-00903-1) although their work only addresses parameter-efficient training of networks for image-classification type problems.

1. Hamiltonian NN (https://arxiv.org/abs/1906.01563) (see 🤦 P.S.)

2. Symplectic ODE Nets (https://arxiv.org/abs/1909.12077)

3. TRS-ODEN (https://arxiv.org/abs/2007.11362)

4. Hamiltonian OpInf (https://arxiv.org/abs/2107.12996)

are different types of neural network architectures / reduced-order models that attempt to endow models with better inductive biases, in the expectation that the resulting trained model can generalize better and enjoy predictable longer time rollouts.

We will start with the following questions:

a. What is the Hamiltonian? How do we interpret it in terms of conservation laws?

b. What is symplectic integration? How does it tie in to Hamiltonian mechanics?

c. How do we write out the Hamiltonian for different systems of interest?

d. How do we learn it from data without explicit knowledge of its functional form? How do these approaches scale and what are interesting applications?

e. What is the typical improvement seen by enforcing the Hamiltonian in predictions over time?


🤦 P.S. Btw, there's Lagrangian Neural Networks too, though my rudimentary understanding is that this is more for convenience of representation rather than any specific advantage: you can [obtain the Hamiltonian from the Lagrangian](https://physics.stackexchange.com/questions/190471/constructing-lagrangian-from-the-hamiltonian) - see https://greydanus.github.io/2020/03/10/lagrangian-nns . Their [GitHub](https://github.com/MilesCranmer/lagrangian_nns) also helps sum up the marketing nicely if you doubt the claims 😏

|	| Neural Networks  | [Neural ODEs](https://arxiv.org/abs/1806.07366) | [HNNs](https://arxiv.org/abs/1906.01563)  | [DeLaN (ICLR'19)](https://arxiv.org/abs/1907.04490) | LNNs (this work) |
| ------------- |:------------:| :------------:| :------------:| :------------:| :------------:|
| Can model dynamical systems | ✔ | ✔ | ✔ | ✔ | ✔ |
| Learns differential equations | | ✔ | ✔ | ✔ | ✔ |
| Learns exact conservation laws | | | ✔ | ✔ | ✔ |
| Learns from arbitrary coords. |✔ | ✔|| ✔ | ✔ |
| Learns arbitrary Lagrangians | | |  | | ✔ |


$\color{red}{\textrm{Free energy}}$?! - https://arxiv.org/pdf/1706.09010.pdf

### Chapter 1: Through goes Hamilton!

Its an [F1 🏎 joke : )](https://youtu.be/TOWEAIG-OXU?si=QxoLCvvL89ygUxwZ&t=104) -

I took a stab at reading [David Morin's chapter](https://bpb-us-e1.wpmucdn.com/sites.harvard.edu/dist/0/550/files/2023/11/cmchap15.pdf) on this topic -  so we will begin by briefly defining the principles of Newtonian, Lagrangian and Hamiltonian mechanics (_the latter two reinterpreting the former in ways that enabled solving problems in other domains!_)

### Chapter 2: If you thought integration was hard...

### Chapter 3: